In [42]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, measure
import fnmatch
from tqdm import tqdm

#Parameters
images_folder = "fluo_corrected"  
masks_folder = "masks"             
output_folder_masks = "masks_inverted"  
pattern_for_files="*_cp_masks.tif"
os.makedirs(output_folder, exist_ok=True)

In [43]:
def invert_zero_mask(image_path, output_path):
    img = io.imread(image_path, as_gray=True)
    img = np.array(img)
    result = np.where(img > 0, 0, 1).astype(np.uint8)
    io.imsave(output_path, result)

def median_background_brightness(image_path, mask_path, output_txt_path):
    """
    Calculate median background brightness from image using inverted mask.
    Only background regions are considered.
    """
    # Load grayscale image
    intensity_image = io.imread(image_path, as_gray=True)
    intensity_image = np.array(intensity_image)

    # Load inverted mask (background = 1, foreground = 0)
    mask = io.imread(mask_path)
    mask = np.array(mask)

   # Boolean mask for all pixels inside the mask
    background= mask != 0

# Compute one median over all those pixels
    median_val = np.median(intensity_image[background])

# Save result
    with open(output_txt_path, "w") as f:
        f.write(f"{median_val}\n")

def process_masks_and_measure(images_root, masks_root, output_root, pattern):
    # Step 1: Invert masks and save to output_root
    for filename in os.listdir(masks_root):
        if fnmatch.fnmatch(filename.lower(), pattern.lower()):
            mask_path = os.path.join(masks_root, filename)

            # Build inverted mask filename
            name, ext = os.path.splitext(filename)
            inverted_mask_filename = f"{name}_inverted{ext}"
            inverted_mask_path = os.path.join(output_root, inverted_mask_filename)

            # Invert mask
            invert_zero_mask(mask_path, inverted_mask_path)

    # Step 2: Loop over inverted masks and compute median background brightness
    for filename in os.listdir(output_root):
        if filename.lower().endswith(".tif") and "_inverted" in filename:
            inverted_mask_path = os.path.join(output_root, filename)

            # Match to original image
            base_name = "corrected_"+filename.replace("_cp_masks_inverted", "")
            possible_images = [f for f in os.listdir(images_root) if f.startswith(base_name)]
            if not possible_images:
                print(f"Warning: No matching image for inverted mask {filename}")
                continue
            image_path = os.path.join(images_root, possible_images[0])

            # Output text file for medians
            median_txt_path = os.path.join(
                output_root, f"{base_name}_median_intensity_background.txt"
            )

            # Compute median background brightness
            median_background_brightness(image_path, inverted_mask_path, median_txt_path)

# Run
process_masks_and_measure(images_folder, masks_folder, output_folder_masks, pattern_for_files)


/tmp/ipykernel_4169435/2723270220.py:5: UserWarning: masks_inverted/10%005_cp_masks_inverted.tif is a low contrast image
  io.imsave(output_path, result)
/tmp/ipykernel_4169435/2723270220.py:5: UserWarning: masks_inverted/10%006_cp_masks_inverted.tif is a low contrast image
  io.imsave(output_path, result)
/tmp/ipykernel_4169435/2723270220.py:5: UserWarning: masks_inverted/10%007_cp_masks_inverted.tif is a low contrast image
  io.imsave(output_path, result)
/tmp/ipykernel_4169435/2723270220.py:5: UserWarning: masks_inverted/10%008_cp_masks_inverted.tif is a low contrast image
  io.imsave(output_path, result)
/tmp/ipykernel_4169435/2723270220.py:5: UserWarning: masks_inverted/10%009_cp_masks_inverted.tif is a low contrast image
  io.imsave(output_path, result)
/tmp/ipykernel_4169435/2723270220.py:5: UserWarning: masks_inverted/20%010_cp_masks_inverted.tif is a low contrast image
  io.imsave(output_path, result)
/tmp/ipykernel_4169435/2723270220.py:5: UserWarning: masks_inverted/20%011_c